In [52]:
import pandas as pd
import numpy as np
import os
import math 

dir_finegrained = "/Volumes/Extreme SSD/Data - Location/Hummingbird_Location_Data/F_Fine_grained_mobility"
finegrained_file = "fine_grained{}.txt"

def read_fine_grained(fine_grained_location,file_number):
    df = pd.read_csv(fine_grained_location,
                     sep="|", skiprows=0,
                     header=0, encoding='ISO-8859-1')
    df = df.drop(['Unnamed: 0', 'Unnamed: 5'], axis=1)
    df = df.rename(columns=lambda x: x.strip())
    for i in df.columns:
        df[i] = df[i].astype(str)
    df = df.apply(lambda x: x.str.strip())
    df = df.iloc[1:, :]

    for i in df.columns[1:4]:
        df[i] = df[i].astype(int)        
    df = infer_datetime(df,file_number)
    
    return df

def infer_datetime(df,file_number):
    # get the expected month from the first entry
    expected_month = math.ceil(file_number/2)

    # try to convert the first entry of time with the specified format
    try:
        dt = pd.to_datetime(df.iloc[0,0], format='%Y-%m-%d %H')
        if dt.month == expected_month:
            print('Datetime format is correct')
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H')
        else:
            raise ValueError
    except ValueError:
        # if it fails, then try automatic inference
        print('Datetime format is incorrect, trying automatic inference')
        try:
            dt = pd.to_datetime(df.iloc[0,0])
            if dt.month == expected_month:
                print('Automatic inference is successful')
                df['time'] = pd.to_datetime(df['time'])
            else:
                raise ValueError
        except ValueError:
            print('Automatic inference failed')

    return df



def read_tower_data(tower_location):

    tower = pd.read_csv(
        tower_location,
        sep="|",
        header=0, encoding='ISO-8859-1')
    tower = tower.drop(['Unnamed: 0', 'Unnamed: 4'], axis=1)
    tower = tower.iloc[1:, :]
    tower = tower.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    tower = tower.rename(columns=lambda x: x.strip())
    tower = tower.rename(columns={'matcher': 'site_id'})
    tower['site_id'] = tower['site_id'].astype(int)
    #print('There are {} cell towers in the dataset'.format(df.site_id.nunique()))
    return tower

def customer_signals_analysis(df):
    df['time'] = pd.to_datetime(df['time'])
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['is_night'] = df['hour'].isin(range(18, 24)) | df['hour'].isin(range(0, 7)) 
    df['is_day'] = (df['hour'].isin(range(7,18))).astype(int)
    df['is_23'] = (df['hour'] == 23).astype(int)
    df['is_other_times'] = (df['hour'] != 23).astype(int)
    df['day'] = df['time'].dt.date
    df['site_count'] = df.groupby('customer_id')['site_id'].transform('nunique')
    df['day_count'] = df.groupby('customer_id')['day'].transform('nunique')

    customers_analysis = df.groupby('customer_id').agg({
        'time': 'count',
        'is_23': 'sum',
        'is_other_times': 'sum',
        'is_weekend': 'sum',
        'is_night': 'sum',
        'is_day': 'sum',
        'day_count': 'first',
        'site_count': 'first'
    }).rename(columns={
        'time': 'signal_count',
        'is_23': 'signal_at_23',
        'is_other_times': 'signal_at_other_times',
        'is_weekend': 'signal_on_weekend',
        'is_night': 'signal_at_night',
        'is_day': 'signal_during_day',
        'day_count': 'unique_days_count',
        'site_count': 'unique_sites_count'
    })

    return customers_analysis.reset_index()

def filter_customers(cust_df, unique_days_threshold, signal_23_ratio_threshold, weekend_signal_threshold, night_signal_threshold, day_signal_threshold):
    filtered_df = cust_df[
        (cust_df['unique_days_count'] >= unique_days_threshold) &
        (cust_df['signal_at_23'] / cust_df['signal_count'] <= signal_23_ratio_threshold) &
        (cust_df['signal_on_weekend'] > weekend_signal_threshold) &
        (cust_df['signal_at_night'] > night_signal_threshold) &
        (cust_df['signal_during_day'] > day_signal_threshold)
    ]
    return filtered_df


In [33]:
fine_name = finegrained_file.format(6)
fine_path = os.path.join(dir_finegrained, fine_name)

In [34]:
df = read_fine_grained(fine_path,6)

/var/folders/t6/jrc9ppsd2xl0pp4pxgr93x3h0000gn/T/ipykernel_40953/1315678895.py:10: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fine_grained_location,


Datetime format is correct


In [53]:
cust_df = customer_signals_analysis(df)

In [54]:
filtered_df = filter_customers(cust_df, unique_days_threshold=10, signal_23_ratio_threshold=0.5, weekend_signal_threshold=5, night_signal_threshold=10, day_signal_threshold=10)

,index,time,customer_id,segment,site_id,hour,day_of_week,is_weekend,is_night,is_day,is_23,is_other_times,day,site_count,day_count
0,1,2020-03-25 10:00:00,524413568,1,2657,10,2,0,False,1,0,1,2020-03-25,3,16
1,2,2020-03-25 14:00:00,524413568,1,2657,14,2,0,False,1,0,1,2020-03-25,3,16
2,3,2020-03-25 15:00:00,524413568,1,2657,15,2,0,False,1,0,1,2020-03-25,3,16
3,4,2020-03-25 17:00:00,524413568,1,31042,17,2,0,False,1,0,1,2020-03-25,3,16
4,5,2020-03-25 18:00:00,524413568,1,2657,18,2,0,True,0,0,1,2020-03-25,3,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21923568,22510913,2020-03-30 23:00:00,69957112,14,21064,23,0,0,True,0,1,0,2020-03-30,18,16
21923569,22510914,2020-03-30 23:00:00,69957112,14,27319,23,0,0,True,0,1,0,2020-03-30,18,16
21923570,22510915,2020-03-30 10:00:00,422028680,14,8036,10,0,0,False,1,0,1,2020-03-30,10,16
21923571,22510916,2020-03-30 11:00:00,422028680,14,8036,11,0,0,False,1,0,1,2020-03-30,10,16
